In [1]:
import numpy as np

2D数据（长×宽）输入

In [2]:
h = 32 # 输入数据的高度
w = 48 # 输入数据的宽度
input_2Ddata = np.random.randn(h, w) 
output_2Ddata = np.zeros(shape=(h, w)) # 卷积输出尺寸与输入一样


In [3]:
kern = np.random.randn(3, 3) # 3×3卷积核
# kern = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64) # sobel卷积核

In [4]:
padding = np.zeros(shape=(h+2, w+2)) # 0填充
padding[1:-1, 1:-1] = input_2Ddata
for i in range(h):
    for j in range(w):
        window = padding[i:i+3, j:j+3] # 中心像素(i,j)的局部窗口
        output_2Ddata[i, j] = np.sum(kern*window) # 卷积运算即内积

3D数据（长×宽×通道）输入

In [2]:
def conv2D(input_2Ddata, kern):
    (h, w) = input_2Ddata.shape # 输入数据的高度和宽度
    (kern_h, kern_w) = kern.shape # 卷积核的高度和宽度
    padding_h = (kern_h-1)//2
    padding_w = (kern_w-1)//2
    padding = np.zeros(shape=(h+2*padding_h, w+2*padding_w)) # 0填充
    padding[padding_h:-padding_h, padding_w:-padding_w] = input_2Ddata    
    output_2Ddata = np.zeros(shape=(h, w)) # 输出数据的尺寸和输入数据一样
    
    for i in range(h):
        for j in range(w):
            window = padding[i:i+kern_h, j:j+kern_w] # 局部窗口
            output_2Ddata[i,j] = np.sum(kern*window) # 内积
    return output_2Ddata

In [3]:
h = 32 # 输入数据的高度
w = 48 # 输入数据的宽度
in_d = 12 # 输入数据的深度
out_d = 24 # 输出数据的深度
input_3Ddata = np.random.randn(h, w, in_d) 
output_3Ddata = np.zeros(shape=(h, w, out_d))

In [4]:
(kern_h, kern_w) = (3, 3) # 或者(5, 5)
kerns = np.random.randn(out_d, kern_h, kern_w, in_d) # 4D卷积核
bias = np.random.randn(out_d) # 1D偏置

In [5]:
for m in range(out_d): # 每一个输出2D数据
    for k in range(in_d): # 每一个输入2D数据
        input_2Ddata = input_3Ddata[:,:, k] # 第k个输入2D数据
        kern = kerns[m, :,:, k]
        output_3Ddata[:,:, m] += conv2D(input_2Ddata, kern) # 加上每个卷积结果
    output_3Ddata[:,:, m] += bias[m] # 每个输出2D数据只有一个偏置

3D数据（长×宽×通道）输入 2.0

In [6]:
def conv2D(input_2Ddata, kern, in_size, out_size, kern_size=3, stride=1):
    (h1, w1) = in_size # 输入数据尺寸
    (h2, w2) = out_size # 输出数据尺寸
    
    output_2Ddata = np.zeros(shape=out_size)
    
    for i2,i1 in zip(range(h2), range(0, h1, stride)): # 输入数据进行步长
        for j2,j1 in zip(range(w2), range(0, w1, stride)):
            window = input_2Ddata[i1:i1+kern_size, j1:j1+kern_size] # 局部窗口
            output_2Ddata[i2, j2] = np.sum(kern*window) # 内积
    return output_2Ddata

In [7]:
h1 = 32 # 输入数据高度
w1 = 48 # 输入数据宽度
d1 = 12 # 输入数据深度
input_3Ddata = np.random.randn(h1, w1, d1) 

In [8]:
S = 2 # 步长
F = 3 # 卷积核尺寸
d2 = 24 # 输出数据深度

In [9]:
P = (F-1)//2 # 填充尺寸
h2 = (h1-F+2*P)//S + 1 # 输出数据高度
w2 = (w1-F+2*P)//S + 1 # 输出数据宽度
padding = np.zeros(shape=(h1+2*P, w1+2*P, d1)) # 0填充
padding[P:-P, P:-P, :] = input_3Ddata
output_3Ddata = np.zeros(shape=(h2, w2, d2))
kerns = np.random.randn(d2, F, F, d1) # 4D卷积核
bias = np.random.randn(d2) # 1D偏置

In [10]:
for m in range(d2): # 每个输出2D数据
    for k in range(d1): # 每个输入2D数据
        input_2Ddata = padding[:,:, k] # 第k个输入2D数据
        kern = kerns[m, :,:, k] # 卷积核
        output_3Ddata[:,:, m] += conv2D(input_2Ddata, kern, 
        in_size=(h1, w1), out_size=(h2, w2), kern_size=F, stride=S) # 加上每个卷积结果
        
    output_3Ddata[:,:, m] += bias[m] # 每个输出2D数据只有一个偏置